In [1]:
import matplotlib as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import cv2 as cv

In [2]:
img_h,img_w=180,180

In [3]:
batch_size=50
data_dir="Y:\College\Technical Seminar\Iris Dataset\Combined\Clarkson Combined";

In [4]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    label_mode='categorical',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3021 files belonging to 3 classes.
Using 2417 files for training.


In [5]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode='categorical',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3021 files belonging to 3 classes.
Using 604 files for validation.


In [2]:

resnet_model=Sequential()
pretrained_model=tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180,180,3),
    pooling='avg',
    classes=3,
    weights='imagenet'
)
for layer in pretrained_model.layers:
  layer.trainable=False


resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512,activation='relu'))
resnet_model.add(Dense(3,activation='softmax'))



In [4]:
import pandas as pd
layers = [(layer.name, layer.trainable) for layer in pretrained_model.layers]
df = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', None)
print(df)

              Layer Name  Layer Trainable
0                input_1            False
1              conv1_pad            False
2             conv1_conv            False
3               conv1_bn            False
4             conv1_relu            False
5              pool1_pad            False
6             pool1_pool            False
7    conv2_block1_1_conv            False
8      conv2_block1_1_bn            False
9    conv2_block1_1_relu            False
10   conv2_block1_2_conv            False
11     conv2_block1_2_bn            False
12   conv2_block1_2_relu            False
13   conv2_block1_0_conv            False
14   conv2_block1_3_conv            False
15     conv2_block1_0_bn            False
16     conv2_block1_3_bn            False
17      conv2_block1_add            False
18      conv2_block1_out            False
19   conv2_block2_1_conv            False
20     conv2_block2_1_bn            False
21   conv2_block2_1_relu            False
22   conv2_block2_2_conv          

In [7]:
print(resnet_model.layers[0].summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 180, 180, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 186, 186, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 90, 90, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 90, 90, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [8]:
set_trainable = False
for layer in resnet_model.layers[0].layers:
    if layer.name == 'conv5_block3_3_conv' or layer.name=='conv5_block3_3_bn':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
resnet_model.layers[0].summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 180, 180, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 186, 186, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 90, 90, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 90, 90, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [10]:
features=resnet_model.layers[-2].output
print(features)
keras.models.Model(inputs=resnet_model.input, outputs=features)

KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='module_wrapper_1/dense/Relu:0', description="created by layer 'module_wrapper_1'")


In [14]:
from tensorflow.keras.metrics import Precision, Recall
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy',Precision(),Recall()])


In [15]:
epochs=10
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_iris.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]
history =resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)



Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/10
49/49 [==============================] - 130s 3s/step - loss: 0.2737 - accuracy: 0.9102 - precision: 0.9197 - recall: 0.9053 - val_loss: 0.1739 - val_accuracy: 0.9321 - val_precision: 0.9321 - val_recall: 0.9321
Epoch 2/10
49/49 [==============================] - 124s 3s/step - loss: 0.0597 - accuracy: 0.9768 - precision: 0.9772 - recall: 0.9764 - val_loss: 0.0427 - val_accuracy: 0.9818 - val_precision: 0.9818 - 

In [11]:
import cv2
image=cv2.imread("D://Iris Images//Testing_Clarkson//0831097_Live_I1_2_0071_L_Dalsa.bmp")
image_resized=cv2.resize(image,(img_h,img_w))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [17]:
img_dir = "D:\\Iris Images\\Dataset\\2015-20180801T100551Z-001\\2015\\Iris\\LivDet-Iris-2015-Clarkson-Training\\LivDet-Iris-2015-Clarkson-Training\\Dalsa-Live-Training"
import os
img_files = os.listdir(img_dir)
features_list = []

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import load_img, img_to_array
for img_file in img_files:
    # Load the image and preprocess it
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(img_h,img_w))
    image=np.expand_dims(image_resized,axis=0)
    features=resnet_model.layers[-3].output
    feature=keras.models.Model(inputs=resnet_model.input,outputs=features).predict(image)
    features_list.append(feature)
# feature=keras.models.Model(inputs=resnet_model.input, outputs=features).predict(image)
# print(feature.shape)

In [ ]:
# image=cv2.imread("D:\\Iris Images\\Combined_Clarkson\\Live\\0074238_Live_I1_0055_R.bmp")
# image_resized=cv2.resize(image,(img_h,img_w))
# image=np.expand_dims(image_resized,axis=0)
# features=resnet_model.layers[-3].output
# feature=keras.models.Model(inputs=resnet_model.input,outputs=features).predict(image)
# feature.shape

In [19]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "D:\\Libraries(Python)\\OpenCV\\dalsa.xlsx"
df.to_excel(excel_file)

In [ ]:
restnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_h,img_w,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=73728))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()